In [1]:
from torch.onnx.symbolic_opset9 import tensor
from torchvision.transforms import ToTensor, Resize, Compose
from torch.utils.data import DataLoader
import torch
from pytorch_ood.utils import ToRGB
from gtsrb import GTSRB
from pathlib import Path
from PIL import Image
import pillow_heif
import torchvision.transforms.functional as F

/opt/anaconda3/envs/mls/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/anaconda3/envs/mls/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trans = Compose([
            ToRGB(),
            ToTensor(),
            Resize((64, 64), antialias=True)
        ])


batch_size = 5
test_data = GTSRB(root=".", train=False, transforms=trans)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

resnet18_model = torch.load("64x64/label-net-resnet18-64.pt", map_location="cpu", weights_only=False)
wideresnet40_model = torch.load("64x64/label-net-wrn40-64.pt", map_location="cpu", weights_only=False)

#png_image_folder = Path("phys_imgs/png")
png_image_folder = Path("output/crops")
resnet18_model.eval()
wideresnet40_model.eval()


pillow_heif.register_heif_opener()  # activate HEIC-Support

In [4]:
#img = Image.open(f"{heic_img_folder}/IMG_9567.HEIC")
#img = img.resize((64, 64))
#img.show()

images = []
to_tensor = ToTensor()
for img in sorted(png_image_folder.glob("*.jpg")):
    img = Image.open(img)
    img = img.resize((64, 64))
    img_t = to_tensor(img)
    images.append(img_t)

images = torch.stack(images)
images.shape

torch.Size([15, 3, 64, 64])

In [5]:
def classify_images(imgs, model):

    with torch.no_grad():
        pred = model(imgs)

    ergs = pred.argmax(1).tolist()
    return ergs

In [6]:
labels = [1,1,1,38,38,38,33,33,33,11,11,11,16,16,16]

In [7]:
def chunks(lst, n=3):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

ergs = classify_images(images, wideresnet40_model)
b = []
for i in range(len(ergs)):
    b.append(ergs[i]==labels[i])

l = chunks(labels)
e = chunks(ergs)

print("nah => mittel => fern")
print(f"{l}     # True Labels")
print(f"{e}     # Predicted Labels")
print()
print(b)

c = 0
for x in b:
    if x:
       c += 1
erg = c/len(b)
print(f"Accuracy: {round(erg,2)}")

nah => mittel => fern
[[1, 1, 1], [38, 38, 38], [33, 33, 33], [11, 11, 11], [16, 16, 16]]     # True Labels
[[1, 1, 1], [38, 38, 38], [33, 33, 33], [11, 11, 11], [16, 16, 16]]     # Predicted Labels

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Accuracy: 1.0
